<a href="https://colab.research.google.com/github/sandrakaku/ml0930/blob/master/mlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tensorflow.keras.datasets.mnist import load_data
# ((訓練圖片, 訓練答案), (側視圖片, 測試答案))
(x_train, y_train), (x_test, y_test) = load_data()

In [ ]:
print(x_train.shape)
print(x_test.shape)

In [ ]:
import random 
import matplotlib.pyplot as plt
import pandas as pd
idx = random.randint(0, 59999)
plt.imshow(x_train[idx], cmap="gray") #加_r r=reverse 顏色反轉
pd.DataFrame(x_train[idx]) 

In [ ]:
from tensorflow.keras.models import Sequential #model大廈 Sequential樓層 的概念
from tensorflow.keras.layers import Dense #全連接(每一層都跟下一層有關係)
layers = [
    # 784 * 128 + 128(bias)
    Dense(256, activation="relu", input_dim=784), #28*28=784 #最後可以調調看
    # 128 * 10 + 10(bias)
    Dense(10, activation="softmax")
]
model = Sequential(layers)
model.summary()

In [ ]:
# x:scaling 0-255 -> 0-1 框架(tf)隨機係數的關係
# y:one-hot encoding 十個答案
from tensorflow.keras.utils import to_categorical

# x_train.reshape(-1, 784) #-1可以代表不知道請電腦自己代
x_train_norm = x_train.reshape(-1, 784) / 255 #框架的關係
x_test_norm = x_test.reshape(-1, 784) / 255
y_train_cat = to_categorical(y_train, 10)
y_test_cat = to_categorical(y_test, 10)
print(y_train[0], y_train_cat[0])
# x_train_norm[0]

In [ ]:
# 一個神經元: binary crossentropy p log 1/q + (1-p) log 1/(1-q)
# 多個神經元: categorical crossentropy p1 log 1/q1 +......
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
model.compile(loss=CategoricalCrossentropy(), #編譯的步驟
              optimizer=Adam(), #無腦選
              metrics=["accuracy"]) #多給一些額外的衡量標準 回歸要選R2

In [ ]:
# batch: 100~200(小圖片) 10~20(大圖片)
# epoch: 整份資料要看幾遍
# 60000筆, 2epoch, 200batch -> 60000 * 2 / 200次調整(梯度調整)
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
callbacks = [
    ModelCheckpoint("model.h5", save_best_only=True),
    EarlyStopping(patience=5, restore_best_weights=True)
]
model.fit(x_train_norm,
     y_train_cat,
     batch_size=100,
     epochs=50, # epochs訓練的次數
     validation_split=0.1,
     callbacks=callbacks,
     verbose=2) #不顯示進度條

In [ ]:
#最後一次測試
model.evaluate(x_test_norm, y_test_cat)